In [2]:
import requests

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
toronto_neighborhood_page = requests.get(wikipedia_link)

In [5]:
page = toronto_neighborhood_page.text

## Using HTMLParser library and adding custom functions

In [7]:
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
    _data = []
    _attr = ('xxx', 'zzzzz')
    _loc_url = []
    _postal = []
    _borough = []
    _neighborhood = []
    _na = []
    _b = 0
    _n = 0
    _l = 'e'
    _le = 'n' 
    m_dup = 0
    _open = 0
    
    def handle_starttag(self, tag, attrs):
        #print( "Start tag:", tag)
        #self._le = self._l
        for attr in attrs:
            if attr[1].find('/wiki/') != -1:
                #print('ATTR[1] WIKI DETECTED')
                self._attr = attr
            #print("     attr:", attr, ' type :', type(attr))

    def handle_endtag(self, tag):
        self._attr = ('xxx', 'zzzzz')
        #print ("End tag  :", tag)

    def handle_data(self, data):
        #self._data.append(data)
        #print('B : ', self._b, ' DATAM b4 :', data )
       
        if data.find('M') != -1 and len(data) == 3:
           #print('DATAM DETECTED')
            self._le = data
            #print(' SLEF._LE', self._le)
            if len(self._postal) !=0:
                if data != self._postal[len(self._postal) - 1]:                
                    self.m_dup = 0     
                    #print('BOROUGH COMPARE B4 : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                else:
                    self.m_dup = 1
                    self._b = 1
                    #print('BOROUGH COMPARE : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                    if self._borough[len(self._borough) - 1] != data:
                        self._b = 1
                        
                
            
        if self._b == 1 and data.find('Not assigned') != -1:
            self._neighborhood.append(self._borough[len(self._borough) - 1])
            self._b = 0
        if self._b == 1 and data.find('Not assigned') == -1 and len(data) > 3 and self._attr[1].find('zzzz') != -1:
            #print('UNLINKED DATA DETECTED', data, ' test self._attr[1]: ', self._attr[1])
            if self.m_dup != 0:
                self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                self._b = 0
            else:  
                self._neighborhood.append(data)
                self._b = 0                    
                
        if data == 'North York':
            self._open = 1
            
        if self._attr[1].find('/wiki/') != -1 and self._open==1:
            #print(' M B4 LOGIC : ', self.m_dup)
            if self._b == 0 and self.m_dup == 0:
                #print('B = 0 DETECTED', " M = 0", ' le :', self._le)
                self._postal.append(self._le)
                self._borough.append(data)
                self._b = 1    
            elif self._b == 1 and self.m_dup == 0:
                #print('B = 1 DETECTED')
                self._neighborhood.append(data)
                self._b = 0
            elif self._b == 1 and self.m_dup == 1:
                if data != self._borough[len(self._borough) - 1]:
                    #print('DUPLICATE IDENTIFIED NEIGH: ', data,  ' b or n: ', self._borough[len(self._borough) - 1] )
                    self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                    self._b = 0
                    
                
            
        if data == 'South of Bloor':
            self._open = 0   

parser = MyHTMLParser()
parser.feed(page)

In [10]:
import pandas as pd
t_columns = ['PostalCode','Borough','Neighborhood']
n_df = pd.DataFrame(columns=t_columns)
n_df[t_columns[0]] = parser._postal
n_df[t_columns[1]] = parser._borough
n_df[t_columns[2]] = parser._neighborhood
n_df.head(101)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North\n
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District\n"


In [11]:
n_df.shape

(101, 3)